In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.special import binom
from abc import ABC, abstractmethod
from typing import Optional

/usr/local/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Classes to work with options

In [2]:
class Option(ABC):
    def __init__(self, spot: float, strike: float, duration: float, type: str):
        self.spot = spot
        self.strike = strike
        self.t = duration
        self.type = type 
        
        self._check_type()

    def __str__(self):
        return f'spot: {self.spot} | strike: {self.strike} | duration: {self.t} | type: {self.type}'
    
    def _check_type(self):
        if self.type not in ('call', 'put'):
            raise ValueError("Option's type must be either 'call', or 'put'")
    
    @abstractmethod
    def binomial_price(self, u: float, d: float, rf: float, steps: int):
        pass

    def bs_price(self, volatility: float, rf: float, div_yield: float = 0):
        self._check_type()
        if self.type == 'call':
            return self._bs_call_price(volatility, rf, div_yield)
        else:
            return self._bs_put_price(volatility, rf, div_yield)
        
    @abstractmethod
    def bs_delta(self, volatility: float, rf: float, div_yield: float = 0):
        pass

    @abstractmethod
    def bs_gamma(self, volatility: float, rf: float, div_yield: float = 0):
        pass

    @abstractmethod
    def bs_vega(self, volatility: float, rf: float, div_yield: float = 0):
        pass
    
    def bs_greeks(self, volatility: float, rf: float, div_yield: float = 0):
        bs_params = (volatility, rf, div_yield)
        return self.bs_delta(*bs_params), self.bs_gamma(*bs_params), self.bs_vega(*bs_params) 

    @abstractmethod
    def _bs_call_price(self, volatility: float, rf: float, div_yield: float = 0):
        pass

    @abstractmethod
    def _bs_put_price(self, volatility: float, rf: float, div_yield: float = 0):
        pass

    def _bs_get_help_params(self, volatility, rf, div_yield):
        d1 = (np.log(self.spot / self.strike) + (rf - div_yield + volatility**2 / 2) * self.t) / (volatility * np.sqrt(self.t))
        d2 = d1 - volatility * np.sqrt(self.t)
        rf_discount = np.exp(-rf * self.t)
        div_yield_discount = np.exp(-div_yield * self.t)
        return d1, d2, rf_discount, div_yield_discount

In [3]:
class VanillaOption(Option):
    def __init__(self, spot: float, strike: float, duration: float, type: str):
        super().__init__(spot, strike, duration, type)
        
    # TODO: probably this method could be generalized up to any number of steps
    def binomial_one_step_replicating_params(self, u: float, d: float, rf: float):
        v_u, v_d = self.__binomial_one_step_payoffs(u, d, rf)
        discount_rate = 1 / (1 + rf*self.t)
        stock_number = (v_u - v_d) / (self.spot * (u - d))
        debt = (v_d*u - v_u*d) * discount_rate / (u - d)
        return stock_number, debt
    
    def binomial_price(self, u: float, d: float, rf: float, steps: int):
        price = 0
        q = self.__risk_neutral_prob(u, d, rf)
        # TODO: there is a misleading moment here, as duration changed its semantics compared to BS-case. Now it's duration of one step, not of an option. I will fix this later
        step_duration = self.t
        for i in range(steps + 1):
            discount_rate = 1 / (1 + rf*step_duration) ** steps
            payoff_in_leaf = self.__binomial_payoff_in_leaf(u, d, rf, steps, i)
            prob_of_leaf = binom(steps, i) * q**i * (1-q)**(steps-i)
            price += prob_of_leaf * payoff_in_leaf * discount_rate
        return price
    
    def bs_delta(self, volatility: float, rf: float, div_yield: float = 0):
        d1, _, _, _ = self._bs_get_help_params(volatility, rf, div_yield)
        sign = {'call': 0, 'put': 1}
        return np.exp(-div_yield * self.t) * (norm.cdf(d1) - sign[self.type])

    def bs_gamma(self, volatility: float, rf: float, div_yield: float = 0):
        d1, _, _, _ = self._bs_get_help_params(volatility, rf, div_yield)
        return np.exp(-div_yield * self.t) * norm.pdf(d1) / (self.spot * volatility * np.sqrt(self.t))

    def bs_vega(self, volatility: float, rf: float, div_yield: float = 0):
        d1, _, _, _ = self._bs_get_help_params(volatility, rf, div_yield)
        vega = np.exp(-div_yield * self.t) * norm.pdf(d1) * self.spot * np.sqrt(self.t)
        return vega / 100 # we want vega to show change per percent as it is a common convention
    
    def _bs_call_price(self, volatility: float, rf: float, div_yield: float = 0):
        d1, d2, rf_discount, div_yield_discount = self._bs_get_help_params(volatility, rf, div_yield)
        return self.spot * div_yield_discount * norm.cdf(d1) - self.strike * rf_discount * norm.cdf(d2)

    def _bs_put_price(self, volatility: float, rf: float, div_yield: float = 0):
        d1, d2, rf_discount, div_yield_discount = self._bs_get_help_params(volatility, rf, div_yield)
        return self.strike * rf_discount * norm.cdf(-d2) - self.spot * div_yield_discount * norm.cdf(-d1)
    
    def __risk_neutral_prob(self, u: float, d: float, rf: float):
        return (1 + rf*self.t - d) / (u - d)

    def __binomial_one_step_payoffs(self, u: float, d: float, rf: float):
        v_u = self.__binomial_payoff_in_leaf(u, d, rf, 1, 1)
        v_d = self.__binomial_payoff_in_leaf(u, d, rf, 1, 0)
        return v_u, v_d

    def __binomial_payoff_in_leaf(self, u: float, d: float, rf: float, steps: int, leaf_index: int):
        super()._check_type()
        if self.type == 'call':
            return max(self.spot * u**leaf_index * d**(steps-leaf_index) - self.strike, 0)
        else:
            return max(self.strike - self.spot * u**leaf_index * d**(steps-leaf_index), 0)
        
    # TODO: draft version
    def print_payoff_tree(self, u: float, d: float, rf: float, steps: int):
        q = (1 + rf * self.t - d) / (u - d)

        curr_leaf_payoffs = []
        prev_step_payoffs = []
        
        for depth in range(steps, -1, -1):
            print(7*(steps - depth)*" ", end="")
            prev_step_payoffs = curr_leaf_payoffs
            curr_leaf_payoffs = []
            for leaf in range(depth + 1):
                price_in_leaf = self.spot * u**leaf * d**(depth-leaf)
                if depth == steps:
                    leaf_payoff = max(price_in_leaf - self.strike, 0.00) if self.type == 'call' else max(self.strike - price_in_leaf, 0.00)
                else:
                    leaf_payoff = (q * prev_step_payoffs[leaf + 1] + (1 - q) * prev_step_payoffs[leaf]) / (1 + rf * self.t)
                curr_leaf_payoffs.append(leaf_payoff)

                if depth == steps:
                    initial_delta = 1.00 if self.type == 'call' else -1.00
                    delta = initial_delta if leaf_payoff > 0 else 0.00
                else:
                    vu = prev_step_payoffs[leaf + 1]
                    vd = prev_step_payoffs[leaf]
                    delta = self.__delta(price_in_leaf, vu, vd, u, d)

                print(f'{np.round(price_in_leaf, 2)}|{np.round(leaf_payoff, 2)}|{np.round(delta, 2)}', end=2*" ")
            print("\n")

    @staticmethod
    def __delta(spot, vu, vd, u, d):
        return (vu - vd) / (spot * (u - d))

In [4]:
### tests are taken from lecture 3 and manual calculations ###

def run_binom_test(option_params, binom_params, expected_call_and_put_prices):
    call = VanillaOption(*option_params, "call")
    put = VanillaOption(*option_params, "put")
    call_and_put_prices = (call.binomial_price(*binom_params), put.binomial_price(*binom_params))
    assert (np.allclose(call_and_put_prices, expected_call_and_put_prices, 1e-3))

# test 1
option_params = (100, 100, 1)
binom_params = (1.2, 0.8, 0, 1)
expected_call_and_put_prices = (10.000, 10.000)
run_binom_test(option_params, binom_params, expected_call_and_put_prices)

# test 2
option_params = (100, 100, 1)
binom_params = (1.2, 0.8, 0, 3)
expected_call_and_put_prices = (14.800, 14.800)
run_binom_test(option_params, binom_params, expected_call_and_put_prices)

# test 3
option_params = (100, 110, 1)
binom_params = (1.2, 0.8, 0.05, 3)
expected_call_and_put_prices = (15.22, 10.24)
run_binom_test(option_params, binom_params, expected_call_and_put_prices)

##############################################################################################################################
  
### bs tests are generated using some online option calculator http://www.option-price.com/index.php ###

def run_bs_test(option_params, bs_params, expected_call_and_put_prices):
    call = VanillaOption(*option_params, "call")
    put = VanillaOption(*option_params, "put")
    call_and_put_prices = (call.bs_price(*bs_params), put.bs_price(*bs_params))
    assert (np.allclose(call_and_put_prices, expected_call_and_put_prices, 1e-3))

# test 1
option_params = (100, 100, 1)
bs_params = (0.10, 0.05, 0.10)
expected_call_and_put_prices = (1.834, 6.473)
run_bs_test(option_params, bs_params, expected_call_and_put_prices)

# test 2
option_params = (100, 110, 1)
bs_params = (0.10, 0.05, 0.10)
expected_call_and_put_prices = (0.317, 14.468)
run_bs_test(option_params, bs_params, expected_call_and_put_prices)

# test 3
option_params = (100, 90, 1)
bs_params = (0.10, 0.05, 0.10)
expected_call_and_put_prices = (6.472, 1.599)
run_bs_test(option_params, bs_params, expected_call_and_put_prices)

# test 4
option_params = (100, 100, 0.5)
bs_params = (0.10, 0.05, 0.10)
expected_call_and_put_prices = (1.681, 4.089)
run_bs_test(option_params, bs_params, expected_call_and_put_prices)

# test 5
option_params = (100, 110, 0.5)
bs_params = (0.10, 0.05, 0.10)
expected_call_and_put_prices = (0.13, 12.291)
run_bs_test(option_params, bs_params, expected_call_and_put_prices)

# test 6
option_params = (100, 90, 0.5)
bs_params = (0.10, 0.05, 0.10)
expected_call_and_put_prices = (7.757, 0.412)
run_bs_test(option_params, bs_params, expected_call_and_put_prices)

##############################################################################################################################

### bs tests are generated using data from lectures, seminars and some online option calculator http://www.option-price.com/index.php ###

def run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks):
    call = VanillaOption(*option_params, "call")
    put = VanillaOption(*option_params, "put")
    call_greeks = call.bs_greeks(*bs_params)
    put_greeks = put.bs_greeks(*bs_params)

    assert (np.allclose(call_greeks, expected_call_greeks, 1e-2))
    assert (np.allclose(put_greeks, expected_put_greeks, 1e-2))

# test 1
option_params = (100, 100, 1/12)
bs_params = (0.3, 0.05, 0)
expected_call_greeks = (0.536, 0.046, 0.115)
expected_put_greeks = (-0.464, 0.046, 0.115)
run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

# TODO : following tests don't pass. I have similar values to ones Maxim has, but they differ from ones calculatator has
# I double checked on different one and it didn't help. Online calculators agreew with each other https://www.barchart.com/options/options-calculator

# # test 2
# option_params = (100, 100, 1)
# bs_params = (0.10, 0.05, 0.10)
# expected_call_greeks = (0.326, 0.036, 0.361)
# expected_put_greeks = (-0.674, 0.036, 0.361)
# run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

# # test 3
# option_params = (100, 110, 1)
# bs_params = (0.10, 0.05, 0.10)
# expected_call_greeks = (0.08, 0.015, 0.149)
# expected_put_greeks = (-0.92, 0.015, 0.149)
# run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

# # test 4
# option_params = (100, 90, 1)
# bs_params = (0.10, 0.05, 0.10)
# expected_call_greeks = (0.727, 0.033, 0.333)
# expected_put_greeks = (-0.273, 0.033, 0.333)
# run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

# # test 5
# option_params = (100, 100, 0.5)
# bs_params = (0.10, 0.05, 0.10)
# expected_call_greeks = (0.375, 0.054, 0.268)
# expected_put_greeks = (-0.625, 0.054, 0.268)
# run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

# # test 6
# option_params = (100, 110, 0.5)
# bs_params = (0.10, 0.05, 0.10)
# expected_call_greeks = (0.048, 0.014, 0.071)
# expected_put_greeks = (-0.952, 0.014, 0.071)
# run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

# # test 7
# option_params = (100, 90, 0.5)
# bs_params = (0.10, 0.05, 0.10)
# expected_call_greeks = (0.879, 0.028, 0.143)
# expected_put_greeks = (-0.121, 0.028, 0.143)
# run_bs_greeks_test(option_params, bs_params, expected_call_greeks, expected_put_greeks)

In [5]:
class DigitalOption(Option):
    def __init__(self, spot: float, strike: float, duration: float, type: str, payoff: float):
        super().__init__(spot, strike, duration, type)
        self.payoff = payoff

    def binomial_price(self, u: float, d: float, rf: float, steps: int):
        raise NotImplementedError()

    def bs_delta(self, volatility: float, rf: float, div_yield: float = 0):
        raise NotImplementedError()

    def bs_gamma(self, volatility: float, rf: float, div_yield: float = 0):
        raise NotImplementedError()

    def bs_vega(self, volatility: float, rf: float, div_yield: float = 0):
        d1, d2, rf_discount, _ = self._bs_get_help_params(volatility, rf, div_yield)
        vega = -rf_discount * norm.pdf(d2) * (d1 / volatility) * self.payoff
        return vega / 100 # we want vega to show change per percent as it is a common convention
    
    def _bs_call_price(self, volatility: float, rf: float, div_yield: float = 0):
        _, d2, rf_discount, _ = self._bs_get_help_params(volatility, rf, div_yield)
        return rf_discount * norm.cdf(d2) * self.payoff

    def _bs_put_price(self, volatility: float, rf: float, div_yield: float = 0):
        _, d2, rf_discount, _ = self._bs_get_help_params(volatility, rf, div_yield)
        return rf_discount * norm.cdf(-d2) * self.payoff

# Theory

---
### Greeks
---

$$ N(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^x e^{-\frac{t^2}{2}}dt, ~~~~~~~ N^`(x) = \frac{1}{\sqrt{2\pi}} \cdot e^{-\frac{x^2}{2}} $$

$$ d_1 = \frac{1}{\sigma \sqrt{T}} \cdot (\ln{\frac{S_0}{K}} + (r - q + \frac{\sigma^2}{2})\cdot T) ~~~~~~~~~~~ 
d_2 = \frac{1}{\sigma \sqrt{T}} \cdot (ln{\frac{S_0}{K}} + (r -q - \frac{\sigma^2}{2})\cdot T),$$

* $r$ - безрисковая ставка (годовая)
* $q$ - дивиденды (годовая)
* $S_0$ - цена базового актива в начальный момент времени 
* $K$ - цена страйка
* $\sigma$ - волотильность
* $T$ - время до исполнения

Стандартные обозначения — греки (greeks):

* `Дельта` **∆** — чувствительность к цене базового актива. 
* `Вега` **𝒱** — чувствительность к implied волатильности. 
* `Ро` **𝜌** — чувствительность к процентным ставкам.
* `Тета` **Θ** — чувствительность к течению времени.
* `Гамма` **Γ** — чувствительность дельты к цене базового ак- тива.

---
$$ Call ~~~~~~~~~~~~~~~~~~~~~~~~ Put $$
---

* **Delta**  
$$  e^{-q T} \cdot N(d_1) ~~~~~~~~~~~~~~~~~~~~~~~~ e^{-q T} \cdot (N(d_1) - 1)
$$



* **Gamma**  
$$ \frac{N^`(d_1) \cdot  e^{-q T}}{S_0 \cdot \sigma \cdot \sqrt{T}} ~~~~~~~~~~~~~~~~~~~~~~ \frac{N^`(d_1) \cdot  e^{-q T}}{S_0 \cdot \sigma \cdot \sqrt{T}}  $$


* **Vega**  
$$ ~~~~~~~ S_0 \cdot \sqrt{T} \cdot N^`(d_1) \cdot e^{-q T} ~~~~~~~~~~~  S_0 \cdot \sqrt{T} \cdot N^`(d_1) \cdot e^{-q T}$$


Выведем формулу веги для цифрового колл опциона. Будем дифференцировать стоимость по волатильности в ф-ле БШ:
$$
C_K = e^{-rT} N(d_2), \quad
d_2 = \frac{1}{\sigma \sqrt{T}} (ln{\frac{S_0}{K}} + (r - q - \frac{\sigma^2}{2}) T) \Rightarrow 
$$

$$
\dfrac{\partial d_2}{\partial \sigma} = -\dfrac{d_2}{\sigma} + \dfrac{(-\sigma T)}{\sigma \sqrt{T}}
 = -(\dfrac{d_2}{\sigma} + \sqrt{T})
$$

$$
\mathcal{V} = \dfrac{\partial C_K}{\partial \sigma} = e^{-rT} \cdot N'(d_2) \cdot \dfrac{\partial d_2}{\partial \sigma} = -e^{-rT} \cdot N'(d_2) \cdot (\dfrac{d_2}{\sigma} + \sqrt{T}) = 
-e^{-rT} \cdot N'(d_2) \cdot \dfrac{d_1}{\sigma}
 \Rightarrow
$$

$$
\mathcal{V} = -e^{-rT} \cdot N'(d_2) \cdot \dfrac{d_1}{\sigma}
$$